In [1]:
import glob
import time
import numpy as np
np.set_printoptions(suppress=True)

from SmoothSparseWeights_CVXOPT import *

In [2]:
# exampleName="cheburashka"
# poses_folder="poses-2"
# poses_ind=7
# clip_k=4
# handle_num=11


exampleName="cow"
poses_folder="poses-2"
poses_ind=5
clip_k=4
handle_num=15


# exampleName="wolf"
# poses_folder="poses-1"
# poses_ind=2
# clip_k=4
# handle_num=21



weights = {'W_spatial': 0.0}

In [3]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


['./models/cow-all/poses-2/cow-1.obj', './models/cow-all/poses-2/cow-2.obj', './models/cow-all/poses-2/cow-3.obj', './models/cow-all/poses-2/cow-4.obj', './models/cow-all/poses-2/cow-5.obj', './models/cow-all/poses-2/cow-6.obj', './models/cow-all/poses-2/cow-7.obj']
(11666, 28)
(11666, 21)


In [4]:
#### read our output of simplex_hull.py 
results_path="./results_pipeline/"+exampleName+"-"+str(handle_num)+"/result.txt"
endmembers, mixing_weights=load_result(results_path)
### endmembers shape is handle * poses * 12??

# ### this is to make sure the 3*4 or 4*3 error does not happen.
# endmembers=endmembers.reshape((-1, 4, 3))
# endmembers=np.transpose(endmembers, (0,2,1))
# endmembers=endmembers.reshape((len(mixing_weights), -1, 12))


endmembers=endmembers.reshape((len(mixing_weights),-1))
mixing_weights=mixing_weights.T


print (endmembers.shape)
print (mixing_weights.shape)
print (mixing_weights[:3])
mixing_weights_clip, grad_zero_indices=clip_first_k_values(mixing_weights, clip_k)
print (mixing_weights_clip[:3])
print ((col_major(mixing_weights_clip)[grad_zero_indices]).sum()) ### should be zero

(15, 84)
(11666, 15)
[[0.20570602 0.00042219 0.02655054 0.15167153 0.00018981 0.00004767
  0.00030539 0.00002423 0.02001831 0.00000105 0.00102998 0.14445015
  0.00045419 0.44910447 0.00002447]
 [0.21555833 0.0004373  0.02679411 0.16466939 0.00016431 0.00004939
  0.00029969 0.00002367 0.01424946 0.00000119 0.00101018 0.15187666
  0.00048589 0.42435294 0.00002749]
 [0.24024097 0.00040106 0.02997077 0.15734726 0.00013956 0.00004676
  0.00030475 0.00002278 0.01438586 0.00000122 0.00103343 0.11445163
  0.00051674 0.44110425 0.00003297]]
[[0.20570602 0.         0.         0.15167153 0.         0.
  0.         0.         0.         0.         0.         0.14445015
  0.         0.44910447 0.        ]
 [0.21555833 0.         0.         0.16466939 0.         0.
  0.         0.         0.         0.         0.         0.15187666
  0.         0.42435294 0.        ]
 [0.24024097 0.         0.         0.15734726 0.         0.
  0.         0.         0.         0.         0.         0.11445163
  0.  

In [5]:
scale=find_scale(vertices1)
print (scale)

708.6870049193496


In [6]:
recovered_vertices2=recover_vertices(mixing_weights, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

0.07533573276941448


In [7]:
recovered_vertices2=recover_vertices(mixing_weights_clip, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

initials={'x': col_major(mixing_weights_clip.reshape((-1,len(endmembers)))).reshape((-1,1))}
print (initials['x'].shape)

start=time.time()

res=run(mesh0, mesh1_list, weights, endmembers, mixing_weights.ravel(), grad_zero_indices, initials=initials, choice="vertices_error")

res=res.reshape(mixing_weights.shape)
end=time.time()
print("using time: ", end-start)

recovered_vertices2=recover_vertices(res, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

37144.77739542565
(174990, 1)
(11666, 28)
(11666, 21)
11666 7 15
use vertices error as obj
W_spatial 0.0
     pcost       dcost       gap    pres   dres
 0: -6.3316e+03 -2.1169e+05  8e+05  2e+00  3e-13
 1: -6.2758e+03 -8.4052e+04  1e+05  1e-01  4e-13
 2: -5.7427e+03 -5.9721e+04  7e+04  6e-02  3e-13
 3: -4.9631e+03 -2.5080e+04  3e+04  2e-02  1e-13
 4: -4.5649e+03 -1.4879e+04  1e+04  8e-03  7e-14
 5: -4.2041e+03 -1.1233e+04  1e+04  4e-03  4e-14
 6: -4.1000e+03 -9.6992e+03  8e+03  3e-03  3e-14
 7: -4.0087e+03 -8.2052e+03  7e+03  2e-03  3e-14
 8: -3.9539e+03 -6.4503e+03  7e+03  2e-03  2e-14
 9: -3.9224e+03 -4.3366e+03  7e+03  2e-03  3e-14
10: -3.8441e+03  3.3162e+02  8e+03  2e-03  3e-14
11: -3.7579e+03  4.6299e+03  9e+03  2e-03  3e-14
12: -3.7786e+03  1.1491e+04  1e+04  2e-03  3e-14
13: -3.7132e+03  2.1374e+04  1e+04  2e-03  3e-14
14: -2.8631e+03  8.0590e+04  2e+04  2e-03  3e-14
15: -1.0049e+03  1.5600e+05  3e+04  2e-03  3e-14
16:  2.5027e+03  2.5084e+05  4e+04  2e-03  3e-14
17:  6.7170e+0

In [8]:
print (res.reshape((-1, len(endmembers)))[:3]) ### should have same zeros as initial input.
print ((col_major(res)[grad_zero_indices]).sum()) ### should be zero
print ((col_major(res)[grad_zero_indices]).max()) ### should be zero

[[ 0.00000119 -0.         -0.          0.99999616 -0.         -0.
  -0.         -0.         -0.          0.         -0.          0.00000101
  -0.          0.00000165 -0.        ]
 [ 0.00000118 -0.         -0.          0.99999617 -0.         -0.
  -0.         -0.         -0.          0.         -0.          0.000001
  -0.          0.00000165 -0.        ]
 [ 0.00000119 -0.         -0.          0.99999613 -0.         -0.
  -0.         -0.         -0.          0.         -0.          0.00000102
  -0.          0.00000165 -0.        ]]
3.311497641167226e-19
6.397414440916925e-11


In [9]:
print (((1-res.sum(axis=-1))**2).sum()/len(res))
print (res.min(), res.max())

1.6691701017756654e-32
-5.995591330937263e-12 0.9999985752788407


In [10]:
recover_diff= (res.dot(endmembers)-mixing_weights.dot(endmembers))
recover_diff= (recover_diff**2).sum()
print (recover_diff)

61804824306938.73


In [11]:
# #### read gt per vertex transformation 
# groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+"-*.DMAT")
# print(groundtruth_names)

# gt_all=None
# for i in range(len(groundtruth_names)):
#     gt=load_DMAT(groundtruth_names[i])
#     gt=gt.T
#     gt=gt.reshape((-1,4,3))
#     gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
#     if gt_all is None:
#         gt_all=gt
#     else:
#         gt_all=np.hstack((gt_all, gt))
    
# print(gt_all.shape)


In [12]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

['./models/cow-all/cow.DMAT']
(11666, 15)
1.5653958587979453e-28


In [13]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(len(groundtruth_names), -1, 12)

gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) ### is this true?

print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

['./models/cow-all/poses-2/cow-1.Tmat', './models/cow-all/poses-2/cow-2.Tmat', './models/cow-all/poses-2/cow-3.Tmat', './models/cow-all/poses-2/cow-4.Tmat', './models/cow-all/poses-2/cow-5.Tmat', './models/cow-all/poses-2/cow-6.Tmat', './models/cow-all/poses-2/cow-7.Tmat']
(15, 84)
(7, 15, 12)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.998956    0.0322376   0.0323555  -2.24837   ]
 [-0.0456047   0.664913    0.745527   27.1422    ]
 [ 0.00252045 -0.746225    0.665689   53.4901    ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.99999923 -0.00000004  0.00000002]
 [-0.00000004  1.00000031 -0.00000012]
 [ 0.00000002 -0.00000012  0.99999923]]


In [14]:
Vertices=vertices2[:,poses_ind*3:poses_ind*3+3]

## use recovered vertices instead.
recovered_vertices2=recover_vertices(res2, vertices1, endmembers).reshape((len(vertices1),-1))
Vertices=recovered_vertices2[:,poses_ind*3:poses_ind*3+3]


NameError: name 'res2' is not defined

In [ ]:
print (res.sum(axis=0))
print (res.T.dot(Vertices))

res2=res
res2=res.clip(0,1) ### this is important, when estimate handle position later.
print (((1-res2.sum(axis=-1))**2).sum()/len(res2))

print (res2.sum(axis=0))
print (res2.T.dot(Vertices))

In [ ]:
### plot handles corresponding to gt mixing weights, recovered mixing weights, and post-processed mixing weights.
###

eps=1e-30

gt_handle_position=gt_mixing_weights.T.dot(Vertices)/(gt_mixing_weights.T.sum(axis=-1)[...,np.newaxis]+eps)
recovered_handle_position=mixing_weights.T.dot(Vertices)/(mixing_weights.T.sum(axis=-1)[...,np.newaxis]+eps)
cliped_handle_position=mixing_weights_clip.T.dot(Vertices)/(mixing_weights_clip.T.sum(axis=-1)[...,np.newaxis]+eps)
postprocessed_handle_position=res2.T.dot(Vertices)/(res2.T.sum(axis=-1)[...,np.newaxis]+eps)


%matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')

ax.scatter(gt_handle_position[:,0],gt_handle_position[:,1], gt_handle_position[:,2], c='k', s=50)
ax.scatter(recovered_handle_position[:,0],recovered_handle_position[:,1], recovered_handle_position[:,2], c='r', s=50)
# ax.scatter(cliped_handle_position[:,0],cliped_handle_position[:,1], cliped_handle_position[:,2], c='g', s=50)
ax.scatter(postprocessed_handle_position[:,0],postprocessed_handle_position[:,1], postprocessed_handle_position[:,2], c='b', s=50)
ax.scatter(Vertices[:,0],Vertices[:,1], Vertices[:,2], color='y', s=1, edgecolors='none')

plt.show()